#  **1 The QM7 Dataset**

# Loading the Data, Preprocessing, Initial Data Analysis

In [1]:
import scipy

In [2]:
data = scipy.io.loadmat("qm7.mat")

In [3]:
data

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Mon Feb 18 17:12:08 2013',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[[36.858105 ,  2.9076326,  2.907612 , ...,  0.       ,
           0.       ,  0.       ],
         [ 2.9076326,  0.5      ,  0.29672  , ...,  0.       ,
           0.       ,  0.       ],
         [ 2.907612 ,  0.29672  ,  0.5      , ...,  0.       ,
           0.       ,  0.       ],
         ...,
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ],
         [ 0.       ,  0.       ,  0.       , ...,  0.       ,
           0.       ,  0.       ]],
 
        [[36.858105 , 12.599944 ,  2.9019997, ...,  0.       ,
           0.       ,  0.       ],
         [12.599944 , 36.858105 ,  1.4731166, ...,  0.       ,
           0.       ,  0.       ],
         [ 2.9019997,  1.4731166,  0.5      , ...,  0.    

Our starting point is a simple quantum-chemical dataset, the QM7 dataset, which can be downloaded
here:
http://quantum-machine.org/data/qm7.mat
1
(It is provided as a Matlab file, and it can be loaded in Python using the function scipy.io.loadmat.)
The QM7 dataset consists of 7165 organic molecules, each of which is composed of up to 23 atoms. The 3d
coordinates of each atom in each molecule are available in the variable R. It is an array of size 7165×23×3
containing for each molecule and atom a triplet representing the 3d coordinates. The variable Z is an array
of size 7165×23 which gives for each molecule and atom of the molecule the corresponding atomic number.
An atomic number of 1 corresponds to a hydrogen atom (H), the number 6 corresponds to carbon (C), the
numbers 7 and 8 to nitrogen (N) and oxygen (O) respectively, and finally, the number 16 corresponds to
sulfur (S). If the number is zero, then it indicates that there is no atom at this index, and the corresponding
3d coordinate should therefore be ignored. This allows for representing in the same array molecules of
different sizes. In addition to these geometrical features of the molecule, the dataset also provides for
each molecule its atomization energy (computed via quantum-chemical simulation). These atomization
energy values are stored in the variable T, an array of size 7165.


## 1.1 Visualizing Molecules
There are a variety of libraries for rendering molecular geometries with various degrees of sophistication. A quick and dirty approach is to use the scatter plot function of matplotlib, where each point is an
atom (e.g. plotted according to its xy-coordinates and discarding the z-coordinate). Note that bonds are
not provided as part of QM7 because they are strictly speaking not needed to infer chemical properties
(bonds can be derived from atom coordinates). To better visualize the molecule, one can draw connections
between nearby atoms by plotting a line between two atoms if the Euclidean distance between the two is
smaller than a fixed threshold